# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [35]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
print(city_data_df.shape)
city_data_df.head()

(571, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,verkhnyaya inta,65.9811,60.3094,-5.40,89,97,9.64,RU,1675916758
1,1,rikitea,-23.1203,-134.9692,26.75,73,4,6.40,PF,1675916679
2,2,pisco,-13.7000,-76.2167,23.03,83,0,3.09,PE,1675916429
3,3,lavrentiya,65.5833,-171.0000,-24.44,97,100,1.12,RU,1675916389
4,4,bathsheba,13.2167,-59.5167,25.34,78,20,8.75,BB,1675916510


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1.2,
    frame_width = 900,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE 27-32
city_data_ideal_df = city_data_df.loc[(city_data_df['Max Temp'] >= 27) & (city_data_df['Max Temp'] <= 32)] 

# Drop any rows with null values
city_data_ideal_clean_df = city_data_ideal_df.dropna()

# Display sample data
city_data_ideal_clean_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,vaini,-21.2000,-175.2000,27.09,89,75,4.63,TO,1675916333
6,6,carnarvon,-24.8667,113.6333,30.79,53,63,6.73,AU,1675916676
15,15,san juan del cesar,10.7711,-73.0031,27.53,81,16,2.72,CO,1675916767
37,37,saint-philippe,-21.3585,55.7679,27.77,82,67,5.18,RE,1675916792
38,38,souillac,-20.5167,57.5167,28.21,74,75,7.20,MU,1675916603
...,...,...,...,...,...,...,...,...,...,...
538,538,neuquen,-38.9516,-68.0591,27.33,32,0,2.02,AR,1675917114
541,541,srirampur,22.9489,88.0189,27.56,30,1,3.73,IN,1675917116
544,544,lumut,4.2323,100.6298,30.01,70,20,1.54,MY,1675917117
553,553,kattivakkam,13.2167,80.3167,27.05,60,83,2.04,IN,1675917125


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
city_data_ideal_clean_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_ideal_clean_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
5,vaini,TO,-21.2000,-175.2000,89,
6,carnarvon,AU,-24.8667,113.6333,53,
15,san juan del cesar,CO,10.7711,-73.0031,81,
37,saint-philippe,RE,-21.3585,55.7679,82,
38,souillac,MU,-20.5167,57.5167,74,
...,...,...,...,...,...,...
538,neuquen,AR,-38.9516,-68.0591,32,
541,srirampur,IN,22.9489,88.0189,30,
544,lumut,MY,4.2323,100.6298,70,
553,kattivakkam,IN,13.2167,80.3167,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit": 20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lng = row["Lng"]
    lat = row["Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vaini - nearest hotel: Keleti International Resort
carnarvon - nearest hotel: No hotel found
san juan del cesar - nearest hotel: Casa Murillo
saint-philippe - nearest hotel: Le Baril
souillac - nearest hotel: Hotel SkylineInn
geraldton - nearest hotel: Ocean Centre Hotel
bambous virieux - nearest hotel: Otentic Eco Tent Experience
vaitape - nearest hotel: Oa Oa Lodge
phayakkhaphum phisai - nearest hotel: Sapvilai Hotel
busselton - nearest hotel: Broadwater Beach Resort
bengkulu - nearest hotel: Dena Hotel
avarua - nearest hotel: Paradise Inn
collie - nearest hotel: Ridge Motel
sabang - nearest hotel: Sabang Hill
kavieng - nearest hotel: Kavieng Hotel
hithadhoo - nearest hotel: Pebbles Inn
gondanglegi - nearest hotel: No hotel found
faanui - nearest hotel: Oa Oa Lodge
fare - nearest hotel: hôtel Maitai Lapita Village
mathbaria - nearest hotel: No hotel found
meulaboh - nearest hotel: Hotel Meuligou
polewali - nearest hotel: hotel perdana
port blair - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
5,vaini,TO,-21.2000,-175.2000,89,Keleti International Resort
6,carnarvon,AU,-24.8667,113.6333,53,No hotel found
15,san juan del cesar,CO,10.7711,-73.0031,81,Casa Murillo
37,saint-philippe,RE,-21.3585,55.7679,82,Le Baril
38,souillac,MU,-20.5167,57.5167,74,Hotel SkylineInn
...,...,...,...,...,...,...
538,neuquen,AR,-38.9516,-68.0591,32,Hotel Argentina 307
541,srirampur,IN,22.9489,88.0189,30,No hotel found
544,lumut,MY,4.2323,100.6298,70,The Orient Star Resort Lumut
553,kattivakkam,IN,13.2167,80.3167,60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
map_hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1.0,
    frame_width = 800,
    frame_height = 500,
    hover_cols = ['Hotel Name', 'Country'], 
    color = 'City'
)

# Display the map
map_hotel_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)